<a href="https://colab.research.google.com/github/nspiter-creator/PC_Python_2025II/blob/main/CALCULADORAAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CALCULADORA DE MASA PARA COMPUESTOS QUIMICOS



In [ ]:
%pip install periodictable

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.1/811.1 kB 12.2 MB/s eta 0:00:00


In [6]:
import re
from collections import Counter
import periodictable  # Librería con las masas atómicas

# ------------------------------
# Reglas para mostrar al usuario
# ------------------------------
def mostrar_reglas():
    print("=== REGLAS PARA ESCRIBIR FÓRMULAS QUÍMICAS ===")
    print("1. Cada elemento debe empezar con MAYÚSCULA, y puede tener una segunda letra minúscula.")
    print("   Ejemplo: H, He, Fe, Na")
    print("2. Se pueden usar subíndices numéricos enteros después de un elemento o grupo.")
    print("   Ejemplo: H2, SO4")
    print("3. Se permiten paréntesis para agrupar, con multiplicadores.")
    print("   Ejemplo: Fe(OH)3 → Fe:1, O:3, H:3")
    print("4. Se puede escribir un número al inicio como coeficiente global.")
    print("   Ejemplo: 2H2O → H:4, O:2")
    print("5. Solo se permiten paréntesis (), no corchetes [] ni llaves {}.")
    print("6. No se permiten espacios ni símbolos extraños.\n")

# ------------------------------
# Función para extraer coeficiente global
# ------------------------------
def extraer_coeficiente(formula: str):
    i = 0
    while i < len(formula) and formula[i].isdigit():
        i += 1
    if i == 0:
        return 1, formula
    return int(formula[:i]), formula[i:]

# ------------------------------
# Parser recursivo
# ------------------------------
def parse_formula(formula: str) -> dict:
    def parse_recursive(s, i=0):
        elementos = Counter()
        while i < len(s):
            if s[i] == "(":
                subdict, i = parse_recursive(s, i+1)
                num, i = leer_numero(s, i)
                for k, v in subdict.items():
                    elementos[k] += v * num
            elif s[i] == ")":
                return elementos, i+1
            else:
                match = re.match(r"[A-Z][a-z]?", s[i:])
                if not match:
                    raise ValueError(f"Error de sintaxis en {s[i:]}")
                elem = match.group(0)
                i += len(elem)
                num, i = leer_numero(s, i)
                elementos[elem] += num
        return elementos, i

    def leer_numero(s, i):
        num_str = ""
        while i < len(s) and s[i].isdigit():
            num_str += s[i]
            i += 1
        return (int(num_str) if num_str else 1), i

    resultado, _ = parse_recursive(formula, 0)
    return dict(resultado)

# ------------------------------
# Calcular masa molar con periodictable
# ------------------------------
def calcular_masa(elementos: dict) -> float:
    masa_total = 0.0
    for elem, cantidad in elementos.items():
        try:
            elemento_lib = getattr(periodictable, elem)
        except AttributeError:
            raise ValueError(f"Elemento '{elem}' no reconocido en la tabla periódica.")
        masa_elem = elemento_lib.mass
        if masa_elem is None:
            raise ValueError(f"No se encontró la masa para el elemento '{elem}'.")
        masa_total += masa_elem * cantidad
    return masa_total

# ------------------------------
# Función principal
# ------------------------------
def masa_molar(formula: str):
    coef, formula_sin_coef = extraer_coeficiente(formula)
    elementos = parse_formula(formula_sin_coef)
    masa_base = calcular_masa(elementos)
    masa_total = coef * masa_base
    return masa_total, coef, elementos

# ------------------------------
# Programa principal
# ------------------------------
if __name__ == "__main__":
    mostrar_reglas()
    formula = input("Ingrese la fórmula química: ")
    try:
        masa, coef, elementos = masa_molar(formula)
        print("\n✅ Fórmula válida")
        print("Desglose de elementos:", elementos)
        print(f"Masa molar de {formula} = {masa:.3f} g/mol\n")

        # Mostrar masa de cada elemento
        for elem, cantidad in elementos.items():
            elemento_lib = getattr(periodictable, elem)
            masa_elem = elemento_lib.mass
            print(
                f"La masa atómica del elemento {elem} es: {masa_elem:.5f} g/mol, "
                f"y en la fórmula aparece {cantidad} veces → contribuye {masa_elem * cantidad:.5f} g/mol"
            )
    except Exception as e:
        print("\n❌ Error en la fórmula ingresada.")
        print("Motivo:", e)
        print("Por favor, revise que la escritura cumple con las reglas anteriores.")

ModuleNotFoundError: No module named 'periodictable'